In [127]:
# Importing libraries neeeded for the project
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [128]:
#!/usr/bin/env /Applications/MAMP/Library/bin/python

import mysql.connector

config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'ecotourisme',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cursor = cnx.cursor(dictionary=True)

cursor.execute('SELECT pays.nom as pays, s.id_pays, s.annee, s.gpi, t.arriveesTotal as arriveesTotal, t.arriveesAvion, e.pibParHab, eco.co2, eco.GES_hab as gesParHab FROM surete as s, tourisme as t, economie as e, ecologie as eco, pays where pays.id = s.id_pays and s.id_pays = t.id_pays AND t.id_pays = e.id_pays AND e.id_pays = eco.id_pays AND s.annee = t.annee AND t.annee = e.annee AND e.annee = eco.annee AND e.annee=2020;')
results = cursor.fetchall()
 
#cursor.execute('SELECT pays.nom as pays, s.id_pays, AVG(s.gpi) as gpi, AVG(t.arriveesTotal) as arriveesTotal, AVG(t.arriveesAvion) as arriveesAvion, AVG(e.pibParHab) as pibParHab, AVG(eco.co2) as co2 FROM surete as s, tourisme as t, economie as e, ecologie as eco, pays where pays.id = s.id_pays and s.id_pays = t.id_pays AND t.id_pays = e.id_pays AND e.id_pays = eco.id_pays AND s.annee = t.annee AND t.annee = e.annee AND e.annee = eco.annee GROUP BY pays.id;')
#results = cursor.fetchall()
 

data = []
for row in results:
    pays = row['pays']
    id_pays = row['id_pays']
    annee = row['annee']
    gpi = row['gpi']
    arriveesTotal = row['arriveesTotal']
    arriveesAvion = row['arriveesAvion']
    pibParHab = row['pibParHab']
    co2 = row['co2']
    gesParHab = row['gesParHab']
    data.append({'pays': pays,'id_pays': id_pays, 'arriveesTotal': arriveesTotal, 'arriveesAvion': arriveesAvion, 'gpi': gpi, 'pibParHab': pibParHab, 'co2': co2, 'gesParHab': gesParHab, 'annee': annee})

    #print(id_pays, arriveesTotal, arriveesAvion)

cnx.close()

In [129]:
df = pd.DataFrame(data)

#df.head()

In [130]:
# Créer le DataFrame
df = pd.DataFrame(data)

# Normaliser les indicateurs
df['arriveesTotal'] = df['arriveesTotal'] / df['arriveesTotal'].max()
df['arriveesAvion'] = df['arriveesTotal'] / df['arriveesAvion'].min()
df['gpi'] = df['gpi'].min() / df['gpi']
df['pibParHab'] = df['pibParHab'] / df['pibParHab'].max()
df['co2'] = df['co2'].min() / df['co2']
df['gesParHab'] = df['gesParHab'].min() / df['gesParHab']

# Définir les poids pour chaque variable
poids = {
    'arriveesTotal': 2,
    'gpi': 4,
    'pibParHab': 2,
    'gesParHab': 6, 

}

# Calculer le score touristique avec des poids
df['score_touristique'] = df.apply(lambda row: sum(row[var] * poids[var] for var in poids) / sum(poids.values()), axis=1)

# Afficher le DataFrame
df.head()

# afficher la ligne quand id_pays = USA
#df[df['id_pays'] == 'US']

,pays,id_pays,arriveesTotal,arriveesAvion,gpi,pibParHab,co2,gesParHab,annee,score_touristique
0,Angola,AO,0.000546,0.000050,0.575078,0.063827,0.011941,0.143873,2020,0.235164
1,Albanie,AL,0.022697,0.002063,0.621759,0.140266,0.053979,0.346866,2020,0.349583
2,Émirats arabes unis,AE,0.069030,0.006275,0.545500,0.716018,0.001258,0.006467,2020,0.270778
3,Argentine,AR,0.026437,0.002403,0.566222,0.208294,0.001531,0.078488,2020,0.228948
4,Arménie,AM,0.003202,0.000291,0.570026,0.141346,0.035070,0.296185,2020,0.310450


In [131]:
max_score = df['score_touristique'].max()
min_score = df['score_touristique'].min()

# Définir les limites des compartiments
bins = [0, 0.2, 0.25, 0.3, 0.35, max_score]

print(max_score, min_score)


# Définir les labels pour les compartiments
labels = ['E', 'D', 'C', 'B', 'A']

# Créer une nouvelle colonne score
df['score'] = pd.cut(df['score_touristique'], bins=bins, labels=labels)

df = df.sort_values(by='score_touristique', ascending=False)

# Afficher le DataFrame
df.head(50)

# nombre de pays ayant un score A
df['score'].value_counts()


0.5924372967724011 0.15731526644179497


score
D    36
C    34
B    25
A    17
E     7
Name: count, dtype: int64

CLUSTERING 

In [132]:
from sklearn.cluster import KMeans
import numpy as np

# Sélectionner les variables pour le clustering
X = df[['arriveesTotal', 'arriveesAvion', 'gpi', 'pibParHab', 'co2', 'gesParHab']]

# Supprimer les lignes avec des valeurs manquantes et réinitialiser l'index
X = X.dropna().reset_index(drop=True)

# Filtrer df pour correspondre à l'index de X
df = df.loc[X.index]

# Créer le modèle
kmeans = KMeans(n_clusters=5, random_state=42)

# Faire le clustering
kmeans.fit(X)

# Ajouter les labels de clusters dans le DataFrame
df['cluster'] = kmeans.labels_

df = df.sort_values(by='cluster', ascending=False)

centroids = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)

# Afficher les centroïdes
centroids

# Afficher le DataFrame
df.head()




,pays,id_pays,arriveesTotal,arriveesAvion,gpi,pibParHab,co2,gesParHab,annee,score_touristique,score,cluster
118,Afrique du Sud,ZA,0.033191,0.003017,0.467769,0.135610,0.000602,0.068611,2020,0.187167,E,4
18,Bhoutan,BT,0.000256,0.000023,0.778405,0.111601,0.228555,0.296245,2020,0.365343,A,3
54,Islande,IS,0.004167,0.000379,1.000000,0.552081,0.163556,0.083802,2020,0.401094,A,3
55,Israël,IL,0.007574,0.000689,0.416698,0.416397,0.004046,0.049536,2020,0.200854,D,3
40,France,FR,1.000000,0.090909,0.568557,0.446882,0.000886,0.131078,2020,0.425318,A,3


In [133]:
import pycountry

# Fonction pour convertir ISO-2 en ISO-3
def iso2_to_iso3(iso2):
    try:
        return pycountry.countries.get(alpha_2=iso2).alpha_3
    except (AttributeError, LookupError):
        return np.nan
    
# Appliquer la fonction à la colonne 'id_pays'
df['id_pays'] = df['id_pays'].apply(iso2_to_iso3)

# Créer une carte du monde
fig = px.choropleth(df, locations='id_pays', color='cluster', hover_name='pays', projection='natural earth')

# Mettre à jour la carte
fig.update_geos(showcountries=True, countrycolor="Black", showland=True, showocean=True, oceancolor="LightBlue", showlakes=True, lakecolor="Blue")

# Afficher la carte
fig.show()


In [134]:
# Créer une figure
fig = go.Figure()

# Ajouter les points pour chaque cluster
for cluster in df['cluster'].unique():
    cluster_df = df[df['cluster'] == cluster]
    fig.add_trace(go.Scatter3d(x=cluster_df['pibParHab'],
                               y=cluster_df['co2'],
                               z=cluster_df['gesParHab'],
                               mode='markers+text',
                               text=cluster_df['pays'],  # Remplacez 'nom_du_pays' par le nom de la colonne contenant les noms des pays
                               marker=dict(size=4),
                               name=f'Cluster {cluster}'))
    
# Ajouter les centroïdes
fig.add_trace(go.Scatter3d(x=centroids['pibParHab'],
                           y=centroids['gesParHab'],
                           z=centroids['gpi'],
                           mode='markers',
                           marker=dict(size=6, color='black', symbol='x'),
                           name='Centroid'))

# Mettre à jour les axes
fig.update_scenes(xaxis_title='pibParHab',
                  yaxis_title='gesParHab',
                  zaxis_title='GPI')

# Afficher la figure
fig.show()

In [135]:
import plotly.express as px

# Créer un DataFrame avec les colonnes nécessaires
df_plot = df[['pibParHab', 'arriveesTotal', 'cluster', 'pays', 'gesParHab', 'gpi']]

# Créer un graphique de dispersion interactif
df_plot = df[['pibParHab', 'arriveesTotal', 'cluster', 'pays', 'gesParHab', 'gpi']]
fig = px.scatter(df_plot, x='gesParHab', y='cluster', color='cluster', hover_data=['pays', 'gpi', 'arriveesTotal', 'pibParHab'])

# Afficher le graphique
fig.show()